In [1]:
import os
import tensorflow as tf
import RMS_utils

In [2]:
model_HE = tf.keras.models.load_model('Results/Saved Model - H&E')

In [3]:
model_trichrome = tf.keras.models.load_model('Results/Saved Model - Trichrome')

In [4]:
model_MYoD1 = tf.keras.models.load_model('Results/Saved Model - Myod1 .91')

In [5]:
model_Myogenin = tf.keras.models.load_model('Results/Saved Model - Myogenin .90')

In [6]:
#Set trainable to false for models
model_HE.trainable = False
model_trichrome.trainable = False
model_Myogenin.trainable = False
model_MYoD1.trainable = False

#Set input shape and prepare directory
input_shape = [200, 200, 3]
inputs = tf.keras.layers.Input(shape=input_shape)

train_dir, validation_dir, class_weight = RMS_utils.prep_dir('RMS MyoD1')


No. ARMS Train: 90
No. ERMS Train: 225
No. ARMS Validation: 40
No. ERMS Validation: 103


In [7]:
#Creates first combined model
combined_1 = tf.keras.layers.Concatenate()([model_MYoD1(inputs), model_trichrome(inputs)])
outputs_1 = tf.keras.layers.Dense(1, activation='sigmoid')(combined_1)
concat_model_1 = tf.keras.models.Model(inputs, outputs_1)

concat_model_1.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 1)            60106977    input_1[0][0]                    
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 1)            60106977    input_1[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 2)            0           sequential[0][0]                 
                                                                 sequential_3[0][0]    

In [8]:
#Creates second combined model
combined_2 = tf.keras.layers.Concatenate()([model_HE(inputs), model_Myogenin(inputs)])
outputs_2 = tf.keras.layers.Dense(1, activation = 'sigmoid')(combined_2)
concat_model_2 = tf.keras.models.Model(inputs, outputs_2)

concat_model_2.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 1)            56076545    input_1[0][0]                    
__________________________________________________________________________________________________
sequential_6 (Sequential)       (None, 1)            60106977    input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 2)            0           sequential_2[0][0]               
                                                                 sequential_6[0][0]    

In [9]:
#Combines both concatenated models
combined_final = tf.keras.layers.Concatenate()([concat_model_1(inputs), concat_model_2(inputs)])
outputs_final = tf.keras.layers.Dense(1, activation = 'sigmoid')(combined_final)
model_final = tf.keras.models.Model(inputs, outputs_final)

model_final.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
functional_1 (Functional)       (None, 1)            120213957   input_1[0][0]                    
__________________________________________________________________________________________________
functional_3 (Functional)       (None, 1)            116183525   input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 2)            0           functional_1[0][0]               
                                                                 functional_3[0][0]    

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

model_final.compile(optimizer = Adam(lr=0.00009), 
              loss = 'binary_crossentropy', 
              metrics = ['AUC'])

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator, validation_generator = RMS_utils.prep_datagen(train_dir, validation_dir, train_datagen, 50, 200)

Found 315 images belonging to 2 classes.
Found 143 images belonging to 2 classes.


In [11]:
history = model_final.fit(train_generator,
                      validation_data = validation_generator,
                      epochs = 4,
                      verbose = 1,
                      class_weight= class_weight
                   )

Epoch 1/4
7/7 [==============================] - 18s 3s/step - loss: 0.3926 - auc: 0.8241 - val_loss: 0.7199 - val_auc: 0.7008
Epoch 2/4
7/7 [==============================] - 4s 618ms/step - loss: 0.3927 - auc: 0.8230 - val_loss: 0.7197 - val_auc: 0.6875
Epoch 3/4
7/7 [==============================] - 4s 619ms/step - loss: 0.3925 - auc: 0.8257 - val_loss: 0.7196 - val_auc: 0.6903
Epoch 4/4
7/7 [==============================] - 4s 615ms/step - loss: 0.3925 - auc: 0.8248 - val_loss: 0.7195 - val_auc: 0.6823


In [59]:
model_Myogenin.fit(train_generator,
                      validation_data = validation_generator,
                      epochs = 4,
                      verbose = 1,
                      class_weight= class_weight
                   )

Epoch 1/4
31/31 [==============================] - 18s 591ms/step - loss: 0.0485 - auc_3: 0.9932 - val_loss: 0.5617 - val_auc_3: 0.8852
Epoch 2/4
31/31 [==============================] - 17s 561ms/step - loss: 0.0195 - auc_3: 0.9984 - val_loss: 0.4601 - val_auc_3: 0.9176
Epoch 3/4
31/31 [==============================] - 18s 590ms/step - loss: 0.0093 - auc_3: 0.9998 - val_loss: 0.4455 - val_auc_3: 0.9098
Epoch 4/4
31/31 [==============================] - 18s 568ms/step - loss: 0.0032 - auc_3: 1.0000 - val_loss: 0.4648 - val_auc_3: 0.9131


In [13]:
model_MYoD1.fit(train_generator,
                      validation_data = validation_generator,
                      epochs = 4,
                      verbose = 1,
                      class_weight= class_weight
                   )

Epoch 1/4
7/7 [==============================] - 17s 2s/step - loss: 0.0063 - auc_2: 1.0000 - val_loss: 0.5138 - val_auc_2: 0.9090
Epoch 2/4
7/7 [==============================] - 4s 518ms/step - loss: 5.6131e-04 - auc_2: 1.0000 - val_loss: 0.9089 - val_auc_2: 0.8708
Epoch 3/4
7/7 [==============================] - 3s 490ms/step - loss: 0.0046 - auc_2: 1.0000 - val_loss: 0.9187 - val_auc_2: 0.8882
Epoch 4/4
7/7 [==============================] - 4s 532ms/step - loss: 0.0111 - auc_2: 0.9999 - val_loss: 1.2204 - val_auc_2: 0.8943


In [61]:
model_trichrome.fit(train_generator,
                      validation_data = validation_generator,
                      epochs = 4,
                      verbose = 1,
                      class_weight= class_weight
                   )

Epoch 1/4
31/31 [==============================] - 18s 578ms/step - loss: 0.1291 - auc_2: 0.9628 - val_loss: 0.3786 - val_auc_2: 0.9072
Epoch 2/4
31/31 [==============================] - 17s 562ms/step - loss: 0.0384 - auc_2: 0.9971 - val_loss: 0.4530 - val_auc_2: 0.9089
Epoch 3/4
31/31 [==============================] - 17s 562ms/step - loss: 0.0224 - auc_2: 0.9989 - val_loss: 0.5420 - val_auc_2: 0.8816
Epoch 4/4
31/31 [==============================] - 17s 563ms/step - loss: 0.0215 - auc_2: 0.9982 - val_loss: 0.4857 - val_auc_2: 0.9141


In [62]:
model_HE.fit(train_generator,
                      validation_data = validation_generator,
                      epochs = 4,
                      verbose = 1,
                      class_weight= class_weight
                   )

Epoch 1/4
31/31 [==============================] - 18s 574ms/step - loss: 0.0850 - auc: 0.9843 - val_loss: 0.3749 - val_auc: 0.8563
Epoch 2/4
31/31 [==============================] - 18s 566ms/step - loss: 0.0174 - auc: 0.9997 - val_loss: 0.3184 - val_auc: 0.8874
Epoch 3/4
31/31 [==============================] - 18s 565ms/step - loss: 0.0167 - auc: 0.9985 - val_loss: 0.3854 - val_auc: 0.9070
Epoch 4/4
31/31 [==============================] - 17s 564ms/step - loss: 0.0180 - auc: 0.9979 - val_loss: 0.4758 - val_auc: 0.9031


In [14]:
RMS_utils.show_preds(model_MYoD1, validation_generator)

Confusion Matrix
[[37  3]
 [17 86]]
              precision    recall  f1-score   support

        ARMS       0.69      0.93      0.79        40
        ERMS       0.97      0.83      0.90       103

    accuracy                           0.86       143
   macro avg       0.83      0.88      0.84       143
weighted avg       0.89      0.86      0.87       143

